# Welcome to the Lab 🥼🧪
## How do I search markets?

In this notebook, we will develop a basic intuition of how the search, the methods, and how downloading works

The Parcl Labs API includes +70k markets, each with a unique identifier called the parcl_id. The search feature allows you to retrieve these parcl_ids, enabling you to use the rest of the API to pull data about markets.

NOTE: Using the search feature does not count against your credits.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To view the technical docs for this endpoint, click [here](https://docs.parcllabs.com/reference/search_markets_v1_search_markets_get-1)

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/getting_started/search.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs

In [ ]:
import os
import pandas as pd
from parcllabs import ParclLabsClient

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search Markets
Parcl Labs supports over 70,000 housing markets across the United States. This includes:
- Zip Codes
- Cities
- Counties
- Metro Areas

In [ ]:
# Search for a specific market by name and type
# In this case, we are going to search for New York CBSA (Core Based Statistical Area, also known as Metro Area)
market = client.search.markets.retrieve(
    query='New York',
    location_type='CBSA',
)

market

In [ ]:
# The key thing to note here is the parcl_id, which is the unique identifier for the market.
# This allows us to distinguish between different markets with the same name, and have a central language for describing geographic areas. 
# Why can't we use FIPS codes? Boundaries change over time, a parcl_id is unique to a boundary at a point in time. 
# Fips codes can also also start with 0, which can cause issues with some programming languages and data conversions.

print(f"Market Parcl ID: {market['parcl_id'][0]} -- Market Name: {market['name'][0]}")

In [ ]:
# Now lets say you want to do analysis for the entire country
market_df = client.search.markets.retrieve(
    query='United States'
)

market_df

In [ ]:
# Now lets search for New York City, the 5 boroughs of New York
market_df = client.search.markets.retrieve(
    query='New York City',
    location_type='CITY',
)

market_df

In [ ]:
# Now lets search for a specific zip code in New York City
market_df = client.search.markets.retrieve(
    query='10013',
    location_type='ZIP5',
)
market_df

In [ ]:
# Now lets search for all zip codes in New York
market_df = client.search.markets.retrieve(
    state_abbreviation='NY',
    location_type='ZIP5',
    limit=1000,  # expand the default limit of 12 to 1000
    auto_paginate=True # Traverse all results until we have obtained all the zip codes
)
market_df

In [ ]:
# now lets say we know the parcl_id and want to get the market details
market_df = client.search.markets.retrieve(
    parcl_id=2900187, # New York CBSA
)
market_df

In [ ]:
# Now lets say we organize our data interally by fips codes and want to get the market details
# and the parcl_id's to join against external data
fips = [
    '36061', # New York County
    '36047', # Kings County
    '36081', # Queens County
    '36085', # Richmond County
    '36005', # Bronx County
]

all_markets = []

for fip in fips:
    market_df = client.search.markets.retrieve(
        geoid=fip,
    )
    all_markets.append(market_df)

all_markets_df = pd.concat(all_markets)
all_markets_df

In [ ]:
# Now lets say I want the top 100 metros in the country based on population
market_df = client.search.markets.retrieve(
    location_type='CBSA',
    sort_by='TOTAL_POPULATION',
    sort_order='DESC', # most populous first
    limit=100 # truncate to top 100
)

market_df.head(5)

In [ ]:
# same query except with income
market_df = client.search.markets.retrieve(
    location_type='CBSA',
    sort_by='MEDIAN_INCOME',
    sort_order='DESC', # most populous first
    limit=100 # truncate to top 100
)

market_df.head(5)

In [ ]:
# Now lets say you want all cities in EAST_NORTH_CENTRAL Census Region, ordered
# by total_population
market_df = client.search.markets.retrieve(
    region='EAST_NORTH_CENTRAL',
    location_type='CITY',
    limit=1000,  # expand the default limit of 12 to 1000
    sort_by='TOTAL_POPULATION',
    sort_order='DESC', # most populous first
    auto_paginate=True # Traverse all results until we have obtained all the zip codes
)

market_df.head(5)

In [ ]:
# Now lets say you want all markets that we have a daily price feed for
market_df = client.search.markets.retrieve(
    sort_by='PRICEFEED_MARKET',
    limit=100,  # expand the default limit of 12 to 1000
)

market_df.head(5)

In [ ]:
# Now lets say you want all price feed markets that are on the parcl exchange
market_df = client.search.markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    limit=15,  # expand the default limit of 12 to 1000
)

market_df

### 3. Retrieve data about markets

In [ ]:
# Now you have a basic understanding of search. It's very powerful and the entry point to the Parcl Labs ecosystem. 
# It's scalable across all endpoints, for example

# find supply/demand for many markets
supply_demand = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=market_df['parcl_id'].tolist(),
    limit=1 # get most recent
)

supply_demand # note 5826765 corresponds to the entire country